<a href="https://colab.research.google.com/github/yakaralar/IE492/blob/master/weather_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import glob
import os
import requests
import shutil
from tqdm import tqdm_notebook as tqdm

In [0]:
stations=pd.read_csv("https://open.meteostat.net/stations.csv",header=None)
stations

,0,1,2,3,4,5,6,7,8,9
0,PABL0,US,Buckland Airport,65.9667,-161.1500,7.0,PABL,NaN,BKC,NaN
1,01001,NO,Jan Mayen,70.9333,-8.6667,10.0,ENJA,1001.0,NaN,Europe/Oslo
2,01002,NO,Grahuken,79.7833,14.4667,NaN,NaN,1002.0,NaN,Europe/Oslo
3,01003,NO,Hornsund,77.0000,15.5000,10.0,NaN,1003.0,NaN,Europe/Oslo
4,01004,NO,Ny-Alesund Ii,78.9167,11.9333,8.0,ENAS,1004.0,NaN,Europe/Oslo
...,...,...,...,...,...,...,...,...,...,...
30404,D5851,DE,Herdorf,50.7668,7.9735,483.0,NaN,NaN,NaN,Europe/Berlin
30405,D5930,DE,Garmisch-Partenkirchen (Kanker),47.4921,11.1747,920.0,NaN,NaN,NaN,Europe/Berlin
30406,D5958,DE,Glees (Laacher See),50.3996,7.2651,287.0,NaN,NaN,NaN,Europe/Berlin
30407,D9041,DE,Zinnwald,50.7313,13.7529,871.0,NaN,NaN,NaN,Europe/Berlin


In [0]:
stations=stations.loc[~stations[7].isna()].reset_index(drop=True)
stations

,0,1,2,3,4,5,6,7,8,9
0,01001,NO,Jan Mayen,70.9333,-8.6667,10.0,ENJA,1001.0,NaN,Europe/Oslo
1,01002,NO,Grahuken,79.7833,14.4667,NaN,NaN,1002.0,NaN,Europe/Oslo
2,01003,NO,Hornsund,77.0000,15.5000,10.0,NaN,1003.0,NaN,Europe/Oslo
3,01004,NO,Ny-Alesund Ii,78.9167,11.9333,8.0,ENAS,1004.0,NaN,Europe/Oslo
4,01006,NO,Edgeoya,78.2333,22.7833,NaN,NaN,1006.0,NaN,Europe/Oslo
...,...,...,...,...,...,...,...,...,...,...
16705,26636,LT,Svencionys,221.0000,222.3000,NaN,NaN,26636.0,NaN,NaN
16706,34868,RU,Iki-burul,111.8000,113.0000,NaN,NaN,34868.0,NaN,NaN
16707,37144,RU,Prohladnaya,199.0000,199.9000,NaN,NaN,37144.0,NaN,NaN
16708,03351,GB,Rostherne No 2,35.0000,35.0000,NaN,NaN,3351.0,NaN,Europe/London


In [0]:
tr_stations=stations.loc[stations[1]=="TR"].reset_index(drop=True)
tr_stations

,0,1,2,3,4,5,6,7,8,9
0,17022,TR,Zonguldak,41.4500,31.8000,136.0,LTAS,17022.0,ONQ,Europe/Istanbul
1,17024,TR,Inebolu,41.9833,33.7833,64.0,NaN,17024.0,NaN,Europe/Istanbul
2,17026,TR,Sinop,42.0333,35.1667,32.0,NaN,17026.0,NaN,Europe/Istanbul
3,17029,TR,Samsun,41.2667,36.3000,162.0,LTAQ,17029.0,SSX,Europe/Istanbul
4,17030,TR,Samsun,41.2833,36.3333,4.0,NaN,17030.0,NaN,Europe/Istanbul
...,...,...,...,...,...,...,...,...,...,...
130,17297,TR,Datca / Karg? Yaylas?,36.7000,27.6667,10.0,NaN,17297.0,NaN,Europe/Istanbul
131,17302,TR,Antalya-bolge / Kaleici,36.8667,30.7000,45.0,NaN,17302.0,NaN,Europe/Istanbul
132,17310,TR,Alanya / Cikcilli,36.5500,32.0000,6.0,NaN,17310.0,NaN,Europe/Istanbul
133,17355,TR,Osmaniye / Kesmeburun,37.1667,36.1667,120.0,NaN,17355.0,NaN,Europe/Istanbul


In [0]:
tr_stations=tr_stations[[0,2,3,4]]
tr_stations.columns=["id","city","lat","lon"]

In [0]:
tr_stations

,id,city,lat,lon
0,17022,Zonguldak,41.4500,31.8000
1,17024,Inebolu,41.9833,33.7833
2,17026,Sinop,42.0333,35.1667
3,17029,Samsun,41.2667,36.3000
4,17030,Samsun,41.2833,36.3333
...,...,...,...,...
130,17297,Datca / Karg? Yaylas?,36.7000,27.6667
131,17302,Antalya-bolge / Kaleici,36.8667,30.7000
132,17310,Alanya / Cikcilli,36.5500,32.0000
133,17355,Osmaniye / Kesmeburun,37.1667,36.1667


In [0]:
beg_date="2015-12-31"
end_date="2020-04-02"

hourly_stats={}
api_key="hidden because of privacy"
for id_ in tqdm(tr_stations["id"].values):
    url="https://api.meteostat.net/v1/history/hourly?station={station_id}&start={start_time}&end={ending_time}&key={api_key}".format(station_id=id_,start_time=beg_date,ending_time=end_date,api_key=api_key)
    req=requests.get(url)
    weather_df=pd.DataFrame.from_dict(req.json()["data"])
    weather_df
    weather_df["id"]=id_
    hourly_stats[id_]=weather_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [0]:
hourly_weather_stats=pd.concat(hourly_stats.values())
hourly_weather_stats

,time,temperature,dewpoint,humidity,precipitation,precipitation_3,precipitation_6,snowdepth,windspeed,peakgust,winddirection,pressure,condition,id
0,2015-12-31 00:00:00,-1.1,-6.8,65.0,NaN,None,None,NaN,13.0,NaN,10.0,1026.8,NaN,17022
1,2015-12-31 01:00:00,-1.2,-6.9,65.0,NaN,None,None,NaN,16.6,NaN,360.0,1026.8,NaN,17022
2,2015-12-31 02:00:00,-2.6,-4.8,85.0,NaN,None,None,NaN,14.8,NaN,30.0,1026.9,NaN,17022
3,2015-12-31 03:00:00,-3.6,-4.9,91.0,NaN,None,None,NaN,7.6,NaN,90.0,1026.7,NaN,17022
4,2015-12-31 04:00:00,-2.6,-4.9,84.0,NaN,None,None,NaN,7.6,NaN,50.0,1026.3,NaN,17022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16339,2020-04-02 19:00:00,7.5,5.5,87.0,0,None,None,NaN,3.7,NaN,86.0,1012.8,2,17031
16340,2020-04-02 20:00:00,7.1,5.6,90.0,0,None,None,NaN,3.7,NaN,85.0,1012.3,3,17031
16341,2020-04-02 21:00:00,8.5,6.3,86.0,0,None,None,NaN,4.0,NaN,330.0,1011.5,3,17031
16342,2020-04-02 22:00:00,6.6,5.2,91.0,0,None,None,NaN,1.8,NaN,114.0,1011.3,3,17031


In [0]:
# 54 den 38 e düştü worth
available_ids=hourly_weather_stats.loc[hourly_weather_stats["time"].str.startswith("2015-12-31 00:")]["id"].values
len(available_ids)

38

In [0]:
hourly_weather_stats=hourly_weather_stats.loc[hourly_weather_stats["id"].isin(available_ids)].reset_index(drop=True)

In [0]:
hourly_weather_stats=hourly_weather_stats.rename(columns={"time":"date"})
hourly_weather_stats['date']=pd.to_datetime(hourly_weather_stats['date'])

In [0]:
dates=pd.date_range(beg_date,end_date, freq='1H')
dates

DatetimeIndex(['2015-12-31 00:00:00', '2015-12-31 01:00:00',
               '2015-12-31 02:00:00', '2015-12-31 03:00:00',
               '2015-12-31 04:00:00', '2015-12-31 05:00:00',
               '2015-12-31 06:00:00', '2015-12-31 07:00:00',
               '2015-12-31 08:00:00', '2015-12-31 09:00:00',
               ...
               '2020-04-01 15:00:00', '2020-04-01 16:00:00',
               '2020-04-01 17:00:00', '2020-04-01 18:00:00',
               '2020-04-01 19:00:00', '2020-04-01 20:00:00',
               '2020-04-01 21:00:00', '2020-04-01 22:00:00',
               '2020-04-01 23:00:00', '2020-04-02 00:00:00'],
              dtype='datetime64[ns]', length=37297, freq='H')

In [0]:
hourly_weather_stats["rounded_date"] = hourly_weather_stats['date'].dt.round('H')
hourly_weather_stats

,date,temperature,dewpoint,humidity,precipitation,precipitation_3,precipitation_6,snowdepth,windspeed,peakgust,winddirection,pressure,condition,id,rounded_date
0,2015-12-31 00:00:00,-1.1,-6.8,65.0,NaN,None,None,NaN,13.0,NaN,10.0,1026.8,NaN,17022,2015-12-31 00:00:00
1,2015-12-31 01:00:00,-1.2,-6.9,65.0,NaN,None,None,NaN,16.6,NaN,360.0,1026.8,NaN,17022,2015-12-31 01:00:00
2,2015-12-31 02:00:00,-2.6,-4.8,85.0,NaN,None,None,NaN,14.8,NaN,30.0,1026.9,NaN,17022,2015-12-31 02:00:00
3,2015-12-31 03:00:00,-3.6,-4.9,91.0,NaN,None,None,NaN,7.6,NaN,90.0,1026.7,NaN,17022,2015-12-31 03:00:00
4,2015-12-31 04:00:00,-2.6,-4.9,84.0,NaN,None,None,NaN,7.6,NaN,50.0,1026.3,NaN,17022,2015-12-31 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335014,2020-04-02 19:00:00,7.0,5.0,87.0,None,None,None,None,13.0,None,210.0,1010,None,17241,2020-04-02 19:00:00
1335015,2020-04-02 20:00:00,8.0,4.9,81.0,None,None,None,None,11.2,None,230.0,1010,None,17241,2020-04-02 20:00:00
1335016,2020-04-02 21:00:00,7.5,3.5,76.0,None,None,None,None,11.2,None,290.0,1009.6,None,17241,2020-04-02 21:00:00
1335017,2020-04-02 22:00:00,6.0,3.0,81.0,None,None,None,None,22.3,None,340.0,1011,None,17241,2020-04-02 22:00:00


In [0]:
hourly_weather_stats.drop(columns=["date"]).groupby("rounded_date")

,temperature,dewpoint,humidity,precipitation,precipitation_3,precipitation_6,snowdepth,windspeed,peakgust,winddirection,pressure,condition,id,city,lat,lon,rounded_date
0,-1.1,-6.8,65.0,NaN,None,None,NaN,13.0,NaN,10.0,1026.8,NaN,17022,Zonguldak,41.45,31.8000,2015-12-31 00:00:00
1,-1.2,-6.9,65.0,NaN,None,None,NaN,16.6,NaN,360.0,1026.8,NaN,17022,Zonguldak,41.45,31.8000,2015-12-31 01:00:00
2,-2.6,-4.8,85.0,NaN,None,None,NaN,14.8,NaN,30.0,1026.9,NaN,17022,Zonguldak,41.45,31.8000,2015-12-31 02:00:00
3,-3.6,-4.9,91.0,NaN,None,None,NaN,7.6,NaN,90.0,1026.7,NaN,17022,Zonguldak,41.45,31.8000,2015-12-31 03:00:00
4,-2.6,-4.9,84.0,NaN,None,None,NaN,7.6,NaN,50.0,1026.3,NaN,17022,Zonguldak,41.45,31.8000,2015-12-31 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335014,7.0,5.0,87.0,None,None,None,None,13.0,None,210.0,1010,None,17241,Isparta / Gölba??,37.85,30.3667,2020-04-02 19:00:00
1335015,8.0,4.9,81.0,None,None,None,None,11.2,None,230.0,1010,None,17241,Isparta / Gölba??,37.85,30.3667,2020-04-02 20:00:00
1335016,7.5,3.5,76.0,None,None,None,None,11.2,None,290.0,1009.6,None,17241,Isparta / Gölba??,37.85,30.3667,2020-04-02 21:00:00
1335017,6.0,3.0,81.0,None,None,None,None,22.3,None,340.0,1011,None,17241,Isparta / Gölba??,37.85,30.3667,2020-04-02 22:00:00


In [0]:
temp_humid_wind_data=hourly_weather_stats[["rounded_date","id","temperature","dewpoint","humidity","windspeed"]]
temp_humid_wind_data=temp_humid_wind_data.sort_values(by="rounded_date").reset_index(drop=True)

In [0]:
temp_humid_wind_data=pd.pivot_table(temp_humid_wind_data,index="rounded_date",columns="id",values=["temperature","dewpoint","humidity","windspeed"])
temp_humid_wind_data.columns

MultiIndex([( 'dewpoint', '17022'),
            ( 'dewpoint', '17038'),
            ( 'dewpoint', '17060'),
            ( 'dewpoint', '17082'),
            ( 'dewpoint', '17086'),
            ( 'dewpoint', '17090'),
            ( 'dewpoint', '17092'),
            ( 'dewpoint', '17096'),
            ( 'dewpoint', '17098'),
            ( 'dewpoint', '17112'),
            ...
            ('windspeed', '17244'),
            ('windspeed', '17260'),
            ('windspeed', '17280'),
            ('windspeed', '17282'),
            ('windspeed', '17290'),
            ('windspeed', '17295'),
            ('windspeed', '17300'),
            ('windspeed', '17350'),
            ('windspeed', '17352'),
            ('windspeed', '17370')],
           names=[None, 'id'], length=152)

In [0]:
trans_dict=tr_stations[["id","city"]].set_index("id").to_dict()["city"]

In [0]:
temp_humid_wind_data.columns=[i[0]+"_"+trans_dict[i[1]] for i in temp_humid_wind_data.columns]

In [0]:
temp_humid_wind_data

,dewpoint_Zonguldak,dewpoint_Trabzon,dewpoint_Istanbul / Ataturk,dewpoint_Merzifon,dewpoint_Tokat,dewpoint_Sivas,dewpoint_Erzincan,dewpoint_Erzurum,dewpoint_Kars,dewpoint_Canakkale,dewpoint_Bandirma,dewpoint_Murted Tur-Afb,dewpoint_Eskisehir,dewpoint_Ankara / Esenboga,dewpoint_Ankara / Etimesgut,dewpoint_Balikesir,dewpoint_Van,dewpoint_Akhisar,dewpoint_Usak,dewpoint_Afyon,dewpoint_Kayseri / Erkilet,dewpoint_Malatya / Erhac,dewpoint_Elazig,dewpoint_Izmir / Cigli,dewpoint_Izmir / Adnan Menderes,dewpoint_Aydin,dewpoint_Isparta,dewpoint_Isparta / Gölba??,dewpoint_Konya,dewpoint_Gaziantep,dewpoint_Diyarbakir,dewpoint_Batman,dewpoint_Bodrum,dewpoint_Dalaman,dewpoint_Antalya,dewpoint_Adana / Incirlik,dewpoint_Adana / Sakirpasa,dewpoint_Iskenderun,humidity_Zonguldak,humidity_Trabzon,...,temperature_Adana / Sakirpasa,temperature_Iskenderun,windspeed_Zonguldak,windspeed_Trabzon,windspeed_Istanbul / Ataturk,windspeed_Merzifon,windspeed_Tokat,windspeed_Sivas,windspeed_Erzincan,windspeed_Erzurum,windspeed_Kars,windspeed_Canakkale,windspeed_Bandirma,windspeed_Murted Tur-Afb,windspeed_Eskisehir,windspeed_Ankara / Esenboga,windspeed_Ankara / Etimesgut,windspeed_Balikesir,windspeed_Van,windspeed_Akhisar,windspeed_Usak,windspeed_Afyon,windspeed_Kayseri / Erkilet,windspeed_Malatya / Erhac,windspeed_Elazig,windspeed_Izmir / Cigli,windspeed_Izmir / Adnan Menderes,windspeed_Aydin,windspeed_Isparta,windspeed_Isparta / Gölba??,windspeed_Konya,windspeed_Gaziantep,windspeed_Diyarbakir,windspeed_Batman,windspeed_Bodrum,windspeed_Dalaman,windspeed_Antalya,windspeed_Adana / Incirlik,windspeed_Adana / Sakirpasa,windspeed_Iskenderun
rounded_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-12-31 00:00:00,-6.8,-0.1,-1.9,-5.2,-4.9,-12.7,-10.1,-8.5,-9.0,-10.1,-3.2,-14.4,-7.7,-13.8,-12.2,-3.4,-7.5,-4.4,-11.7,-11.9,-8.2,-9.5,-7.5,-7.6,-6.7,-11.8,-12.8,-11.8,-13.8,-8.7,0.3,1.0,-8.1,-15.1,-14.3,-8.0,-8.7,-0.9,65.0,81.0,...,7.8,11.6,13.0,16.6,25.9,11.2,3.6,3.6,5.4,18.4,0.0,27.7,35.3,3.6,13.0,11.2,13.0,18.4,29.5,9.4,5.4,5.4,13.0,7.6,9.4,25.9,29.5,9.4,11.2,24.1,24.1,3.6,9.4,14.8,14.8,35.3,38.9,22.3,22.3,18.4
2015-12-31 01:00:00,-6.9,0.1,-2.1,-7.0,NaN,-13.0,-12.0,-10.1,-9.9,-7.9,-3.0,-15.1,-9.1,-13.9,-13.0,-3.0,-8.0,NaN,NaN,-12.0,-9.1,-9.1,-6.9,-7.1,-7.1,NaN,-13.6,-11.9,-12.0,-8.1,0.0,1.0,NaN,-12.9,-12.9,-7.8,-9.9,NaN,65.0,81.0,...,8.0,NaN,16.6,22.3,24.1,9.4,NaN,14.8,7.6,3.6,0.0,24.1,31.7,7.6,13.0,3.6,11.2,11.2,18.4,NaN,NaN,7.6,7.6,14.8,13.0,22.3,29.5,NaN,9.4,37.1,20.5,7.6,5.4,13.0,NaN,46.4,35.3,20.5,20.5,NaN
2015-12-31 02:00:00,-4.8,-0.9,-2.1,-7.0,NaN,-13.0,-12.0,-11.1,-11.0,-9.0,-4.1,-13.9,-8.0,-15.0,-13.9,-3.0,-8.0,-4.8,NaN,-12.2,-8.9,-9.1,-8.1,-7.1,-6.0,NaN,-14.0,-11.9,-13.1,-8.1,0.0,1.0,NaN,-14.2,-12.9,-8.2,-9.0,NaN,85.0,81.0,...,8.0,NaN,14.8,16.6,33.5,5.4,NaN,9.4,11.2,3.6,1.8,24.1,35.3,14.8,9.4,7.6,5.4,14.8,13.0,9.4,NaN,5.4,11.2,16.6,5.4,20.5,27.7,NaN,9.4,33.5,25.9,3.6,5.4,11.2,NaN,40.7,35.3,16.6,20.5,NaN
2015-12-31 03:00:00,-4.9,0.2,-2.1,-6.2,-5.2,-11.4,-12.0,-14.9,-12.0,-9.0,-5.0,-14.4,-8.6,-13.7,-12.9,-3.3,-6.9,-5.2,-13.5,-11.2,-8.7,-9.9,-9.0,-7.1,-6.8,-8.8,-14.8,-11.8,-12.6,-7.3,-0.6,1.0,-6.2,-16.1,-15.3,-9.3,-10.3,-1.0,91.0,87.0,...,7.5,10.8,7.6,25.9,35.3,7.6,1.8,0.0,7.6,5.4,5.4,29.5,35.3,7.6,9.4,3.6,3.6,14.8,20.5,5.4,3.6,3.6,5.4,7.6,9.4,20.5,31.7,1.8,9.4,27.7,24.1,1.8,1.8,5.4,14.8,38.9,37.1,22.3,22.3,13.0
2015-12-31 04:00:00,-4.9,0.1,-2.0,-7.0,NaN,-14.0,-11.1,-13.0,-14.0,-9.0,-5.0,-15.0,-9.1,-15.1,-14.0,-3.0,-5.0,-5.6,NaN,-11.4,-10.0,-11.0,-10.1,-7.1,-6.0,NaN,-15.5,-12.9,-13.0,-10.0,0.0,0.0,NaN,-14.2,-14.2,-9.1,-9.8,NaN,84.0,87.0,...,7.0,NaN,7.6,24.1,24.1,0.0,NaN,13.0,9.4,3.6,14.8,27.7,31.7,5.4,9.4,7.6,1.8,16.6,13.0,9.4,NaN,7.6,9.4,5.4,9.4,20.5,29.5,NaN,1.8,24.1,24.1,9.4,0.0,3.6,NaN,44.6,35.3,22.3,18.4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [0]:
temp_humid_wind_data=temp_humid_wind_data.interpolate(method="time")

In [0]:
temp_humid_wind_data

,dewpoint_Zonguldak,dewpoint_Trabzon,dewpoint_Istanbul / Ataturk,dewpoint_Merzifon,dewpoint_Tokat,dewpoint_Sivas,dewpoint_Erzincan,dewpoint_Erzurum,dewpoint_Kars,dewpoint_Canakkale,dewpoint_Bandirma,dewpoint_Murted Tur-Afb,dewpoint_Eskisehir,dewpoint_Ankara / Esenboga,dewpoint_Ankara / Etimesgut,dewpoint_Balikesir,dewpoint_Van,dewpoint_Akhisar,dewpoint_Usak,dewpoint_Afyon,dewpoint_Kayseri / Erkilet,dewpoint_Malatya / Erhac,dewpoint_Elazig,dewpoint_Izmir / Cigli,dewpoint_Izmir / Adnan Menderes,dewpoint_Aydin,dewpoint_Isparta,dewpoint_Isparta / Gölba??,dewpoint_Konya,dewpoint_Gaziantep,dewpoint_Diyarbakir,dewpoint_Batman,dewpoint_Bodrum,dewpoint_Dalaman,dewpoint_Antalya,dewpoint_Adana / Incirlik,dewpoint_Adana / Sakirpasa,dewpoint_Iskenderun,humidity_Zonguldak,humidity_Trabzon,...,temperature_Adana / Sakirpasa,temperature_Iskenderun,windspeed_Zonguldak,windspeed_Trabzon,windspeed_Istanbul / Ataturk,windspeed_Merzifon,windspeed_Tokat,windspeed_Sivas,windspeed_Erzincan,windspeed_Erzurum,windspeed_Kars,windspeed_Canakkale,windspeed_Bandirma,windspeed_Murted Tur-Afb,windspeed_Eskisehir,windspeed_Ankara / Esenboga,windspeed_Ankara / Etimesgut,windspeed_Balikesir,windspeed_Van,windspeed_Akhisar,windspeed_Usak,windspeed_Afyon,windspeed_Kayseri / Erkilet,windspeed_Malatya / Erhac,windspeed_Elazig,windspeed_Izmir / Cigli,windspeed_Izmir / Adnan Menderes,windspeed_Aydin,windspeed_Isparta,windspeed_Isparta / Gölba??,windspeed_Konya,windspeed_Gaziantep,windspeed_Diyarbakir,windspeed_Batman,windspeed_Bodrum,windspeed_Dalaman,windspeed_Antalya,windspeed_Adana / Incirlik,windspeed_Adana / Sakirpasa,windspeed_Iskenderun
rounded_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-12-31 00:00:00,-6.8,-0.1,-1.9,-5.2,-4.9,-12.7,-10.1,-8.5,-9.0,-10.1,-3.2,-14.4,-7.7,-13.8,-12.2,-3.4,-7.5,-4.4,-11.700000,-11.9,-8.2,-9.5,-7.5,-7.6,-6.7,-11.8,-12.8,-11.8,-13.8,-8.7,0.3,1.0,-8.100000,-15.1,-14.3,-8.0,-8.7,-0.900000,65.0,81.0,...,7.8,11.600000,13.0,16.6,25.9,11.2,3.6,3.6,5.4,18.4,0.0,27.7,35.3,3.6,13.0,11.2,13.0,18.4,29.5,9.4,5.400000,5.4,13.0,7.6,9.4,25.9,29.5,9.400000,11.2,24.1,24.1,3.6,9.4,14.8,14.8,35.3,38.9,22.3,22.3,18.400000
2015-12-31 01:00:00,-6.9,0.1,-2.1,-7.0,-5.0,-13.0,-12.0,-10.1,-9.9,-7.9,-3.0,-15.1,-9.1,-13.9,-13.0,-3.0,-8.0,-4.6,-12.300000,-12.0,-9.1,-9.1,-6.9,-7.1,-7.1,-10.8,-13.6,-11.9,-12.0,-8.1,0.0,1.0,-7.466667,-12.9,-12.9,-7.8,-9.9,-0.933333,65.0,81.0,...,8.0,11.333333,16.6,22.3,24.1,9.4,3.0,14.8,7.6,3.6,0.0,24.1,31.7,7.6,13.0,3.6,11.2,11.2,18.4,9.4,4.800000,7.6,7.6,14.8,13.0,22.3,29.5,6.866667,9.4,37.1,20.5,7.6,5.4,13.0,14.8,46.4,35.3,20.5,20.5,16.600000
2015-12-31 02:00:00,-4.8,-0.9,-2.1,-7.0,-5.1,-13.0,-12.0,-11.1,-11.0,-9.0,-4.1,-13.9,-8.0,-15.0,-13.9,-3.0,-8.0,-4.8,-12.900000,-12.2,-8.9,-9.1,-8.1,-7.1,-6.0,-9.8,-14.0,-11.9,-13.1,-8.1,0.0,1.0,-6.833333,-14.2,-12.9,-8.2,-9.0,-0.966667,85.0,81.0,...,8.0,11.066667,14.8,16.6,33.5,5.4,2.4,9.4,11.2,3.6,1.8,24.1,35.3,14.8,9.4,7.6,5.4,14.8,13.0,9.4,4.200000,5.4,11.2,16.6,5.4,20.5,27.7,4.333333,9.4,33.5,25.9,3.6,5.4,11.2,14.8,40.7,35.3,16.6,20.5,14.800000
2015-12-31 03:00:00,-4.9,0.2,-2.1,-6.2,-5.2,-11.4,-12.0,-14.9,-12.0,-9.0,-5.0,-14.4,-8.6,-13.7,-12.9,-3.3,-6.9,-5.2,-13.500000,-11.2,-8.7,-9.9,-9.0,-7.1,-6.8,-8.8,-14.8,-11.8,-12.6,-7.3,-0.6,1.0,-6.200000,-16.1,-15.3,-9.3,-10.3,-1.000000,91.0,87.0,...,7.5,10.800000,7.6,25.9,35.3,7.6,1.8,0.0,7.6,5.4,5.4,29.5,35.3,7.6,9.4,3.6,3.6,14.8,20.5,5.4,3.600000,3.6,5.4,7.6,9.4,20.5,31.7,1.800000,9.4,27.7,24.1,1.8,1.8,5.4,14.8,38.9,37.1,22.3,22.3,13.000000
2015-12-31 04:00:00,-4.9,0.1,-2.0,-7.0,-5.6,-14.0,-11.1,-13.0,-14.0,-9.0,-5.0,-15.0,-9.1,-15.1,-14.0,-3.0,-5.0,-5.6,-13.700000,-11.4,-10.0,-11.0,-10.1,-7.1,-6.0,-8.4,-15.5,-12.9,-13.0,-10.0,0.0,0.0,-6.600000,-14.2,-14.2,-9.1,-9.8,-1.600000,84.0,87.0,...,7.0,10.366667,7.6,24.1,24.1,0.0,1.2,13.0,9.4,3.6,14.8,27.7,31.7,5.4,9.4,7.6,1.8,16.6,13.0,9.4,2.800000,7.6,9.4,5.4,9.4,20.5,29.5,3.300000,1.8,24.1,24.1,9.4,0.0,3.6,14.2,44.6,35.3,22.3,18.4,9.266667
...,...,..

In [0]:
temp_humid_wind_data=temp_humid_wind_data.reset_index()

In [0]:
temp_humid_wind_data=pd.merge(pd.DataFrame(dates,columns=["rounded_date"]),temp_humid_wind_data.groupby("rounded_date").mean(),on="rounded_date",how="left").set_index("rounded_date")
temp_humid_wind_data=temp_humid_wind_data.interpolate(method="time")
temp_humid_wind_data

,dewpoint_Zonguldak,dewpoint_Trabzon,dewpoint_Istanbul / Ataturk,dewpoint_Merzifon,dewpoint_Tokat,dewpoint_Sivas,dewpoint_Erzincan,dewpoint_Erzurum,dewpoint_Kars,dewpoint_Canakkale,dewpoint_Bandirma,dewpoint_Murted Tur-Afb,dewpoint_Eskisehir,dewpoint_Ankara / Esenboga,dewpoint_Ankara / Etimesgut,dewpoint_Balikesir,dewpoint_Van,dewpoint_Akhisar,dewpoint_Usak,dewpoint_Afyon,dewpoint_Kayseri / Erkilet,dewpoint_Malatya / Erhac,dewpoint_Elazig,dewpoint_Izmir / Cigli,dewpoint_Izmir / Adnan Menderes,dewpoint_Aydin,dewpoint_Isparta,dewpoint_Isparta / Gölba??,dewpoint_Konya,dewpoint_Gaziantep,dewpoint_Diyarbakir,dewpoint_Batman,dewpoint_Bodrum,dewpoint_Dalaman,dewpoint_Antalya,dewpoint_Adana / Incirlik,dewpoint_Adana / Sakirpasa,dewpoint_Iskenderun,humidity_Zonguldak,humidity_Trabzon,...,temperature_Adana / Sakirpasa,temperature_Iskenderun,windspeed_Zonguldak,windspeed_Trabzon,windspeed_Istanbul / Ataturk,windspeed_Merzifon,windspeed_Tokat,windspeed_Sivas,windspeed_Erzincan,windspeed_Erzurum,windspeed_Kars,windspeed_Canakkale,windspeed_Bandirma,windspeed_Murted Tur-Afb,windspeed_Eskisehir,windspeed_Ankara / Esenboga,windspeed_Ankara / Etimesgut,windspeed_Balikesir,windspeed_Van,windspeed_Akhisar,windspeed_Usak,windspeed_Afyon,windspeed_Kayseri / Erkilet,windspeed_Malatya / Erhac,windspeed_Elazig,windspeed_Izmir / Cigli,windspeed_Izmir / Adnan Menderes,windspeed_Aydin,windspeed_Isparta,windspeed_Isparta / Gölba??,windspeed_Konya,windspeed_Gaziantep,windspeed_Diyarbakir,windspeed_Batman,windspeed_Bodrum,windspeed_Dalaman,windspeed_Antalya,windspeed_Adana / Incirlik,windspeed_Adana / Sakirpasa,windspeed_Iskenderun
rounded_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-12-31 00:00:00,-6.8,-0.1,-1.9,-5.2,-4.900000,-12.7,-10.1,-8.5,-9.0,-10.100000,-3.2,-14.4,-7.7,-13.8,-12.2,-3.4,-7.5,-4.4,-11.700000,-11.9,-8.2,-9.5,-7.5,-7.6,-6.7,-11.8,-12.8,-11.8,-13.8,-8.7,0.3,1.0,-8.100000,-15.1,-14.3,-8.0,-8.7,-0.900000,65.0,81.0,...,7.8,11.600000,13.0,16.6,25.9,11.2,3.600000,3.6,5.4,18.4,0.0,27.700000,35.3,3.6,13.0,11.2,13.0,18.4,29.5,9.4,5.4,5.4,13.0,7.6,9.4,25.9,29.5,9.400000,11.2,24.1,24.1,3.6,9.4,14.8,14.8,35.3,38.9,22.3,22.3,18.400000
2015-12-31 01:00:00,-6.9,0.1,-2.1,-7.0,-5.000000,-13.0,-12.0,-10.1,-9.9,-7.900000,-3.0,-15.1,-9.1,-13.9,-13.0,-3.0,-8.0,-4.6,-12.300000,-12.0,-9.1,-9.1,-6.9,-7.1,-7.1,-10.8,-13.6,-11.9,-12.0,-8.1,0.0,1.0,-7.466667,-12.9,-12.9,-7.8,-9.9,-0.933333,65.0,81.0,...,8.0,11.333333,16.6,22.3,24.1,9.4,3.000000,14.8,7.6,3.6,0.0,24.100000,31.7,7.6,13.0,3.6,11.2,11.2,18.4,9.4,4.8,7.6,7.6,14.8,13.0,22.3,29.5,6.866667,9.4,37.1,20.5,7.6,5.4,13.0,14.8,46.4,35.3,20.5,20.5,16.600000
2015-12-31 02:00:00,-4.8,-0.9,-2.1,-7.0,-5.100000,-13.0,-12.0,-11.1,-11.0,-9.000000,-4.1,-13.9,-8.0,-15.0,-13.9,-3.0,-8.0,-4.8,-12.900000,-12.2,-8.9,-9.1,-8.1,-7.1,-6.0,-9.8,-14.0,-11.9,-13.1,-8.1,0.0,1.0,-6.833333,-14.2,-12.9,-8.2,-9.0,-0.966667,85.0,81.0,...,8.0,11.066667,14.8,16.6,33.5,5.4,2.400000,9.4,11.2,3.6,1.8,24.100000,35.3,14.8,9.4,7.6,5.4,14.8,13.0,9.4,4.2,5.4,11.2,16.6,5.4,20.5,27.7,4.333333,9.4,33.5,25.9,3.6,5.4,11.2,14.8,40.7,35.3,16.6,20.5,14.800000
2015-12-31 03:00:00,-4.9,0.2,-2.1,-6.2,-5.200000,-11.4,-12.0,-14.9,-12.0,-9.000000,-5.0,-14.4,-8.6,-13.7,-12.9,-3.3,-6.9,-5.2,-13.500000,-11.2,-8.7,-9.9,-9.0,-7.1,-6.8,-8.8,-14.8,-11.8,-12.6,-7.3,-0.6,1.0,-6.200000,-16.1,-15.3,-9.3,-10.3,-1.000000,91.0,87.0,...,7.5,10.800000,7.6,25.9,35.3,7.6,1.800000,0.0,7.6,5.4,5.4,29.500000,35.3,7.6,9.4,3.6,3.6,14.8,20.5,5.4,3.6,3.6,5.4,7.6,9.4,20.5,31.7,1.800000,9.4,27.7,24.1,1.8,1.8,5.4,14.8,38.9,37.1,22.3,22.3,13.000000
2015-12-31 04:00:00,-4.9,0.1,-2.0,-7.0,-5.600000,-14.0,-11.1,-13.0,-14.0,-9.000000,-5.0,-15.0,-9.1,-15.1,-14.0,-3.0,-5.0,-5.6,-13.700000,-11.4,-10.0,-11.0,-10.1,-7.1,-6.0,-8.4,-15.5,-12.9,-13.0,-10.0,0.0,0.0,-6.600000,-14.2,-14.2,-9.1,-9.8,-1.600000,84.0,87.0,...,7.0,10.366667,7.6,24.1,24.1,0.0,1.200000,13.0,9.4,3.6,14.8,27.700000,31.7,5.4,9.4,7.6,1.8,16.6,13.0,9.4,2.8,7.6,9.4,5.4,9.4,20.5,29.5,3.30

In [0]:
temp_humid_wind_data.isna().sum().sum()

0

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
os.listdir("drive/My Drive/colab_datas/IE 492")

['Comparison of methods.pdf',
 '1-s2.0-S1364032114000914-main.pdf',
 '1-s2.0-S0360544209002539-main.pdf',
 '1-s2.0-S1876610211043116-main.pdf',
 '1-s2.0-S0305048311001423-main.pdf',
 'weather_turkey_processed.zip',
 'Meta_L_makaleler',
 'all_data.feather',
 'UECM-15012015-20022020.csv',
 'RealTimeConsumption-01122014-09032020.csv',
 'updated_support-date_based_information.r',
 'ozel_gunler',
 'RealTimeConsumption_unholidayized.csv',
 'RealTimeConsumption_unhol_decomposed.csv',
 'RealTimeConsumption_holiday.csv',
 'HW_preds.csv',
 'kubi_preds',
 'HW_preds_new.csv',
 'arima_preds.csv',
 'arima_preds_datetime.csv',
 'gpw_v4_admin_unit_center_points_population_estimates_rev11_tur.csv',
 'gpw-v4-country-level-summary-rev11.xlsx',
 'gpw-v4-documentation-rev11.pdf',
 'lasso_coeffs.csv',
 'lasso_coeffs_new.csv',
 'kubi_weather_data_20151231-2020-04-02.csv']

In [0]:
temp_humid_wind_data.to_csv("drive/My Drive/colab_datas/IE 492/kubi_temp_humid_wind_data_20151231-2020-04-02.csv")

In [0]:
hourly_weather_stats.to_csv("drive/My Drive/colab_datas/IE 492/kubi_weather_data_20151231-2020-04-02.csv",index=False)

# date rounding

In [0]:
dates_=pd.DataFrame(hourly_weather_stats["time"])
dates_["hour_part"]=dates_["time"].apply(lambda x:x.split(" ")[1])
dates_["time"]=pd.to_datetime(dates_["time"])
dates_

,time,hour_part
0,2015-12-31 00:00:00,00:00:00
1,2015-12-31 01:00:00,01:00:00
2,2015-12-31 02:00:00,02:00:00
3,2015-12-31 03:00:00,03:00:00
4,2015-12-31 04:00:00,04:00:00
...,...,...
1335014,2020-04-02 19:00:00,19:00:00
1335015,2020-04-02 20:00:00,20:00:00
1335016,2020-04-02 21:00:00,21:00:00
1335017,2020-04-02 22:00:00,22:00:00


In [0]:
dates_["hour_part"].value_counts()

06:00:00    58292
09:00:00    58242
12:00:00    58206
07:00:00    57616
10:00:00    57598
            ...  
17:16:00        1
13:28:00        1
08:26:00        1
16:17:00        1
04:53:00        1
Name: hour_part, Length: 300, dtype: int64

In [0]:
dates_=pd.DataFrame(hourly_weather_stats["time"])
dates_["hour_part"]=dates_["time"].apply(lambda x:x.split(" ")[1])
dates_["time"]=pd.to_datetime(dates_["time"])
dates_

In [0]:
dates_['hour'] = dates_['time'].dt.round('H')
dates_

,time,hour_part,hour
0,2015-12-31 00:00:00,00:00:00,2015-12-31 00:00:00
1,2015-12-31 01:00:00,01:00:00,2015-12-31 01:00:00
2,2015-12-31 02:00:00,02:00:00,2015-12-31 02:00:00
3,2015-12-31 03:00:00,03:00:00,2015-12-31 03:00:00
4,2015-12-31 04:00:00,04:00:00,2015-12-31 04:00:00
...,...,...,...
1335014,2020-04-02 19:00:00,19:00:00,2020-04-02 19:00:00
1335015,2020-04-02 20:00:00,20:00:00,2020-04-02 20:00:00
1335016,2020-04-02 21:00:00,21:00:00,2020-04-02 21:00:00
1335017,2020-04-02 22:00:00,22:00:00,2020-04-02 22:00:00


In [0]:
dates_["hour"].value_counts()

6     58854
9     58468
12    58405
8     57847
11    57820
7     57798
10    57790
14    57726
15    57686
13    57657
18    57372
3     56905
21    56414
5     56074
16    55674
17    55533
0     55442
4     53160
2     53086
20    51861
19    51795
1     50860
22    50815
23    49977
Name: hour, dtype: int64